In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as py
%matplotlib inline

In [2]:
focal = pd.read_csv('designYearClass.csv', usecols=['patent_number','year','class'])

citation = pd.read_csv('utility_patent_citations_clean.csv', usecols=['patent_number','class'])

citation_link = pd.read_csv('designCitations.csv', usecols=['patent_number','reference'])

In [3]:
focal.head()

,patent_number,year,class
0,D257752,1980,D19/75
1,D257924,1980,D06/573
2,D258382,1980,D23/214
3,D258383,1980,D23/214
4,D258571,1980,D09/560


In [4]:
citation.head()

,patent_number,class
0,4069641,52/202
1,4069641,52/476
2,4069641,52/717.1
3,4069641,D25/48.7
4,4069641,D25/119


In [5]:
citation_link.head()

,patent_number,reference
0,D257752,4162014
1,D257924,4162014
2,D258382,D253842
3,D258383,D253842
4,D258678,D253842


In [6]:
#select the utility reference patents

util_link = citation_link[~citation_link['reference'].str.contains('D')]

In [7]:
util_link.head()

,patent_number,reference
0,D257752,4162014
1,D257924,4162014
6,D258766,3939620
7,D258766,3955331
8,D258766,3971178


In [8]:
##trying for real
focal_ref = pd.merge(focal,util_link, on='patent_number')

In [9]:
#sanity check
focal_ref[focal_ref.patent_number == 'D380496']

,patent_number,year,class,reference
265717,D380496,1995,D06/570,4162014
265718,D380496,1995,D06/570,4344241
265719,D380496,1995,D19/90,4162014
265720,D380496,1995,D19/90,4344241
265721,D380496,1995,D19/86,4162014
265722,D380496,1995,D19/86,4344241


In [10]:
citation = citation.rename(index=str, columns={"patent_number":"reference"})

citation['reference']=citation['reference'].apply(str)

In [11]:
master_merge = pd.merge(focal_ref, citation, on='reference')

master_merge['class_combo'] = master_merge['class_x'] +'-'+ master_merge['class_y']

master_merge.head()

,patent_number,year,class_x,reference,class_y,class_combo
0,D257752,1980,D19/75,4162014,211/50,D19/75-211/50
1,D257752,1980,D19/75,4162014,211/55,D19/75-211/55
2,D257752,1980,D19/75,4162014,211/126.1,D19/75-211/126.1
3,D257924,1980,D06/573,4162014,211/50,D06/573-211/50
4,D257924,1980,D06/573,4162014,211/55,D06/573-211/55


In [12]:
master_merge['new_combo'] = 0

In [13]:
def check_new_combo(row):
#     combo = row['class_combo']
#     base_year = row['year']
    earlier_years = master_merge[master_merge['year'] < row['year']]
    return not earlier_years['class_combo'].isin([row['class_combo']]).any()
    

In [14]:
master_merge.sort_values('year').reset_index(inplace=True)

In [ ]:
master_merge['new_combo'] = master_merge.apply(check_new_combo, axis=1)

In [15]:
mini_year = master_merge[master_merge.year == 2000]

In [ ]:
mini_year['new_combo'] = mini_year.apply(check_new_combo, axis=1)